In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/2. 연결개별통합.csv")
df_data = pd.read_csv("../data/2. 증가율데이터.csv", encoding='cp949')

# 증가율데이터를 각 지표로 변경하기
df_data = df_data[['회사명', '거래소코드', '회계년도', '매출액(수익)(*)(IFRS)(천원)','매출액(수익)(*)(IFRS연결)(천원)',
                   '유형자산(*)(IFRS)(천원)', '유형자산(*)(IFRS연결)(천원)', 
                   '재고자산(*)(IFRS)(천원)', '재고자산(*)(IFRS연결)(천원)',
                   '비유동자산(*)(IFRS)(천원)','비유동자산(*)(IFRS연결)(천원)',
              '유동자산(*)(IFRS)(천원)', '유동자산(*)(IFRS연결)(천원)',
              '자본(*)(IFRS)(천원)', '자본(*)(IFRS연결)(천원)',
              '자산(*)(IFRS)(천원)', '자산(*)(IFRS연결)(천원)', 
              '당기순이익(총포괄손익)(IFRS)(천원)', '당기순이익(총포괄손익)(IFRS연결)(천원)']]

연결의 NAN값을 개별의 값으로 채우기

In [2]:
for i in range(len(df_data.columns)):
    if i > 3 and i % 2 == 0 :
        df_data.iloc[:,i] = df_data.iloc[:,i].fillna(df_data.iloc[:,i-1])

# 연결데이터 기준으로 드랍시키기
a = df_data[df_data.iloc[:,4].isnull()].index
df_data.drop(a, inplace=True)
df_data.reset_index(drop='index',inplace=True)

매출액증가율

In [3]:
for i in df_data.index:
    if i > 0:
        if df_data.loc[i,"회사명"] == df_data.loc[i-1,"회사명"]:
            if df_data.iloc[:,4][i] == 0 or df_data.iloc[:,4][i-1] == 0:
                df_data.loc[i,"매출액증가율"] = 0
            else:
                df_data.loc[i,"매출액증가율"] = (df_data.iloc[:,4][i]-df_data.iloc[:,4][i-1])/df_data.iloc[:,4][i-1]
        elif df_data.loc[i,"회사명"] != df_data.loc[i-1,"회사명"]:
            df_data.loc[i,"매출액증가율"] = np.NAN

In [4]:
df_data['매출액증가율']

0             NaN
1        0.563969
2       -0.084291
3        0.255100
4        0.115206
           ...   
13678   -0.072642
13679    0.081171
13680    0.068749
13681   -0.093495
13682   -0.237773
Name: 매출액증가율, Length: 13683, dtype: float64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10061 entries, 0 to 10060
Data columns (total 59 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   회사명         10061 non-null  object 
 1   거래소코드       10061 non-null  int64  
 2   회계년도        10061 non-null  object 
 3   상장일         10061 non-null  object 
 4   상장폐지일       774 non-null    object 
 5   소속코드        10061 non-null  int64  
 6   산업명         10061 non-null  object 
 7   총자본증가율      10061 non-null  float64
 8   유형자산증가율     10061 non-null  float64
 9   비유동자산증가율    10061 non-null  float64
 10  유동자산증가율     10061 non-null  float64
 11  재고자산증가율     10061 non-null  float64
 12  자기자본증가율     10061 non-null  float64
 13  매출액증가율      10061 non-null  float64
 14  순이익증가율      10061 non-null  float64
 15  매출액총이익률     10061 non-null  float64
 16  매출액순이익률     10061 non-null  float64
 17  자기자본순이익률    10061 non-null  float64
 18  총자본사업이익률    10061 non-null  float64
 19  총자본순이익률     10061 non-nul

유형자산증가율

In [6]:
# '회사명'이 같고, '유형자산' 데이터 중 하나라도 0인 경우 유형자산증가율을 0으로 설정
df_data['유형자산증가율'] = np.where(
    (df_data['회사명'] == df_data['회사명'].shift(1)) & ((df_data.iloc[:, 6] == 0) | (df_data.iloc[:, 6].shift(1) == 0)),
    0,
    (df_data.iloc[:, 6] - df_data.iloc[:, 6].shift(1)) / df_data.iloc[:, 6].shift(1)
)

# '회사명'이 다른 경우 유형자산증가율을 NaN으로 설정
df_data['유형자산증가율'] = np.where(
    df_data['회사명'] != df_data['회사명'].shift(1),
    np.NAN,
    df_data['유형자산증가율']
)

재고자산증가율

In [7]:
# '회사명'이 같고, '재고자산' 데이터 중 하나라도 0인 경우 재고자산증가율을 0으로 설정
df_data['재고자산증가율'] = np.where(
    (df_data['회사명'] == df_data['회사명'].shift(1)) & ((df_data.iloc[:, 8] == 0) | (df_data.iloc[:, 8].shift(1) == 0)),
    0,
    (df_data.iloc[:, 8] - df_data.iloc[:, 8].shift(1)) / df_data.iloc[:, 8].shift(1)
)

# '회사명'이 다른 경우 재고자산증가율을 NaN으로 설정
df_data['재고자산증가율'] = np.where(
    df_data['회사명'] != df_data['회사명'].shift(1),
    np.NAN,
    df_data['재고자산증가율']
)

비유동자산증가율

In [8]:
# '회사명'이 같고, '비유동자산' 데이터 중 하나라도 0인 경우 비유동자산증가율을 0으로 설정
df_data['비유동자산증가율'] = np.where(
    (df_data['회사명'] == df_data['회사명'].shift(1)) & ((df_data.iloc[:, 10] == 0) | (df_data.iloc[:, 10].shift(1) == 0)),
    0,
    (df_data.iloc[:, 10] - df_data.iloc[:, 10].shift(1)) / df_data.iloc[:, 10].shift(1)
)

# '회사명'이 다른 경우 비유동자산증가율을 NaN으로 설정
df_data['비유동자산증가율'] = np.where(
    df_data['회사명'] != df_data['회사명'].shift(1),
    np.NAN,
    df_data['비유동자산증가율']
)

유동자산증가율

In [9]:
# '회사명'이 같고, '비유동자산' 데이터 중 하나라도 0인 경우 비유동자산증가율을 0으로 설정
df_data['유동자산증가율'] = np.where(
    (df_data['회사명'] == df_data['회사명'].shift(1)) & ((df_data.iloc[:, 10] == 0) | (df_data.iloc[:, 10].shift(1) == 0)),
    0,
    (df_data.iloc[:, 10] - df_data.iloc[:, 10].shift(1)) / df_data.iloc[:, 10].shift(1)
)

# '회사명'이 다른 경우 비유동자산증가율을 NaN으로 설정
df_data['유동자산증가율'] = np.where(
    df_data['회사명'] != df_data['회사명'].shift(1),
    np.NAN,
    df_data['유동자산증가율']
)

자기자본증가율

In [10]:
# '회사명'이 같고, '비유동자산' 데이터 중 하나라도 0인 경우 비유동자산증가율을 0으로 설정
df_data['자기자본증가율'] = np.where(
    (df_data['회사명'] == df_data['회사명'].shift(1)) & ((df_data.iloc[:, 10] == 0) | (df_data.iloc[:, 10].shift(1) == 0)),
    0,
    (df_data.iloc[:, 10] - df_data.iloc[:, 10].shift(1)) / df_data.iloc[:, 10].shift(1)
)

# '회사명'이 다른 경우 비유동자산증가율을 NaN으로 설정
df_data['자기자본증가율'] = np.where(
    df_data['회사명'] != df_data['회사명'].shift(1),
    np.NAN,
    df_data['자기자본증가율']
)

총자본 : 0이 존재 하지 않음

In [11]:
df_data[df_data["자산(*)(IFRS연결)(천원)"]<0]

,회사명,거래소코드,회계년도,매출액(수익)(*)(IFRS)(천원),매출액(수익)(*)(IFRS연결)(천원),유형자산(*)(IFRS)(천원),유형자산(*)(IFRS연결)(천원),재고자산(*)(IFRS)(천원),재고자산(*)(IFRS연결)(천원),비유동자산(*)(IFRS)(천원),...,자산(*)(IFRS)(천원),자산(*)(IFRS연결)(천원),당기순이익(총포괄손익)(IFRS)(천원),당기순이익(총포괄손익)(IFRS연결)(천원),매출액증가율,유형자산증가율,재고자산증가율,비유동자산증가율,유동자산증가율,자기자본증가율


In [12]:
import numpy as np

# '회사명'이 같은 경우 총자본증가율 계산, 그렇지 않으면 NaN 설정
df_data['총자본증가율'] = np.where(
    df_data['회사명'] == df_data['회사명'].shift(1),
    (df_data.iloc[:, 16] - df_data.iloc[:, 16].shift(1)) / df_data.iloc[:, 16].shift(1),
    np.NAN
)

순이익 : -값이 상대적으로 많음

In [13]:
df_data[df_data["당기순이익(총포괄손익)(IFRS연결)(천원)"]<=0]

,회사명,거래소코드,회계년도,매출액(수익)(*)(IFRS)(천원),매출액(수익)(*)(IFRS연결)(천원),유형자산(*)(IFRS)(천원),유형자산(*)(IFRS연결)(천원),재고자산(*)(IFRS)(천원),재고자산(*)(IFRS연결)(천원),비유동자산(*)(IFRS)(천원),...,자산(*)(IFRS연결)(천원),당기순이익(총포괄손익)(IFRS)(천원),당기순이익(총포괄손익)(IFRS연결)(천원),매출액증가율,유형자산증가율,재고자산증가율,비유동자산증가율,유동자산증가율,자기자본증가율,총자본증가율
0,(주)CMG제약,58820,2010/12,11350419.0,11350419.0,10316860.0,10316860.0,2782759.0,2782759.0,13734412.0,...,26775627.0,-6945220.0,-6945220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820,2011/12,17751704.0,17751704.0,9652599.0,9707759.0,2743103.0,2743103.0,12283654.0,...,27305198.0,-1264488.0,-1508282.0,0.563969,-0.059039,-0.014251,-0.128901,-0.128901,-0.128901,0.019778
2,(주)CMG제약,58820,2012/12,16255389.0,16255389.0,9264142.0,9307482.0,2955172.0,2955172.0,9915161.0,...,57108035.0,-6892739.0,-6660149.0,-0.084291,-0.041233,0.077310,-0.164260,-0.164260,-0.164260,1.091471
5,(주)CMG제약,58820,2015/12,27041894.0,27041894.0,17822468.0,17822468.0,5455617.0,5455617.0,30261207.0,...,56532821.0,-3648668.0,-3650091.0,0.188520,-0.053696,0.110422,0.016589,0.016589,0.016589,-0.026443
11,(주)ES큐브,50120,2010/12,34640205.0,43887335.0,1595923.0,3374904.0,757990.0,4166815.0,21539344.0,...,49189060.0,-2165930.0,-1632071.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13655,휴림로봇(주),90710,2018/12,30729831.0,30729831.0,8418337.0,8446292.0,2919917.0,2986180.0,17796516.0,...,45580695.0,-19342811.0,-19026771.0,-0.570212,0.156399,-0.212753,-0.592075,-0.592075,-0.592075,-0.387328
13656,휴림로봇(주),90710,2019/12,17514292.0,17620760.0,8184725.0,8241931.0,3472442.0,3618863.0,16697108.0,...,42734247.0,-10080845.0,-9335043.0,-0.426591,-0.024195,0.211870,-0.053993,-0.053993,-0.053993,-0.062449
13658,휴마시스(주),205470,2017/12,8508328.0,8508328.0,7594444.0,7594444.0,2484499.0,2484499.0,8561424.0,...,28944370.0,-9199634.0,-9199634.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13659,휴마시스(주),205470,2018/12,9657735.0,9657735.0,8022158.0,8022158.0,2695420.0,2695420.0,8289211.0,...,34293774.0,-852810.0,-852810.0,0.135092,0.056319,0.084895,-0.031795,-0.031795,-0.031795,0.184817


In [14]:
# '회사명'이 같고, '순이익' 값이 양수인 경우에만 순이익증가율을 계산
df_data['순이익증가율'] = np.where(
    (df_data['회사명'] == df_data['회사명'].shift(1)) & (df_data.iloc[:, 18] > 0) & (df_data.iloc[:, 18].shift(1) > 0),
    (df_data.iloc[:, 18] - df_data.iloc[:, 18].shift(1)) / df_data.iloc[:, 18].shift(1),
    0
)

# '회사명'이 다른 경우 순이익증가율을 NaN으로 설정
df_data['순이익증가율'] = np.where(
    df_data['회사명'] != df_data['회사명'].shift(1),
    np.NAN,
    df_data['순이익증가율']
)

데이터프레임 합치기

In [15]:
# 증가율 데이터를 백분율 형태로 변환하고, 회사명, 거래소코드, 회계년도를 포함하는 새로운 데이터프레임 생성
df_increase = pd.concat([
    round(df_data[['매출액증가율', '유형자산증가율', '재고자산증가율', '비유동자산증가율', '유동자산증가율', '자기자본증가율', '총자본증가율', '순이익증가율']] * 100, 2),
    df_data[['회사명', '거래소코드', '회계년도']]
], axis=1)

df_increase["회계년도"]= pd.to_datetime(df_increase["회계년도"],format='%Y/%m')
df["회계년도"]= pd.to_datetime(df["회계년도"])

merged_df = pd.merge(df, df_increase, how='left', on=["회사명", "거래소코드", "회계년도"])

In [16]:
df_all = merged_df[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','매출액증가율_y', '유형자산증가율_y',
       '재고자산증가율_y', '비유동자산증가율_y', '유동자산증가율_y', '자기자본증가율_y', '총자본증가율_y','순이익증가율_y', '매출액총이익률', '매출액순이익률', '자기자본순이익률', '총자본사업이익률',
       '총자본순이익률', '1주당매출액', '1주당순이익', '유동비율', '당좌비율', '현금비율', '순운전자본비율',
       '부채비율', '이자보상배율이자비용', '비유동비율', '유동자산구성비율', '자기자본구성비율', '자기자본배율',
       '총자본회전률', '자기자본회전률', '비유동자산회전률', '재고자산회전률', '매출채권회전률', '매입채무회전률',
       '매입채무회전기간', '재고자산회전기간', '매출채권회전기간', '1회전기간', '운전자본회전률', '순운전자본회전률',
       '유형자산회전율', '유동자산회전률', '당좌자산회전률', '부가가치', '총자본투자효율', '설비투자효율', '기계투자효율',
       '부가가치율', '노동소득분배율', '자본분배율', '이윤분배율']]

In [17]:
df_all.columns = df_all.columns.str.replace('y','')
df_all.columns = df_all.columns.str.replace('_','')

In [18]:
missing_sales_growth_indexes = df_all[df_all["매출액증가율"].isnull()].index
df_all = df_all.drop(missing_sales_growth_indexes)
df_all.reset_index(drop=True, inplace=True)

In [19]:
df_all.to_csv("../data/3. 증가율 데이터 정리끝.csv",index=False,encoding='cp949')

In [20]:
print('총기업수 :',len(df_all["회사명"].unique()))
print('정상기업(중복미포함) :',len(df_all[df_all["소속코드"]==5]["회사명"].unique()))
print('상장폐지 기업 :',len(df_all[df_all["소속코드"]==4]["회사명"].unique()))

총기업수 : 1399
정상기업(중복미포함) : 1171
상장폐지 기업 : 171
